# Numerical root finding

<a href="https://www.thetreecenter.com/wp-content/uploads/tree-roots.jpg" target="_blank"><img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/tree-roots.jpg" /> </a>

## PHYS 2600: Scientific Computing

## Lecture 15

## FAQ: Masking an array

In Python, we can generate a mask using a conditional statement on an array. The mask is another array of Booleans where `True`
means the value in the original array satisfies the condition.

In [ ]:
import numpy as np

N = np.arange(10)
mask = N < 5
for n in N:
    print(f"N[{n}] = {N[n]}, mask[{n}] = {mask[n]}")

Using index notation, a mask generates a slice of the array that only includes the values corresponding to `True` in the mask.

In [ ]:
N[mask]

We can also assign new values to the masked elements in the array. We can either set all of the values at once with a single value, or we can set each value individually with another array, as long as the masked array is the same length as the new array.

In [ ]:
N[mask] = 5
print(N)
N[mask] = np.array([4, 3, 2, 1, 0])
print(N)
print(mask)

This works just like assigment via slicing! Note that the mask doesn't change as we change the elements of the array.

## Root finding

Let's put the new Python syntax we've learned to work, and learn how to do __root finding__.  The basic problem is as follows: given the equation

$$
f(x) = 0,
$$

find all values $\{x_r\}$ that satisfy the equation.  (The solutions $\{x_r\}$ are the _roots_ or _zeroes_ of $f(x)$.)

Many problems in physics and math can be framed as root-finding!  Solving any equation $f(x) = g(x)$ is just root-finding $f(x) - g(x) = 0$.  Calculating $\sqrt{a}$ is equivalent to finding the roots of the equation

$$
y^2 - a = 0.
$$

There are many algorithms for root finding; here are three examples.

* __Brute force__ checks every number over a certain interval to see which ones satisfy the equation. 
* __Bisection__ uses a simple math argument to quickly narrow down a single root.
* __Newton-Raphson__ uses the derivative $f'(x)$ to extrapolate to where the nearest root is.  ("Heron's method" for the square root is actually Newton-Raphson!)

All of these methods are __iterative methods__: start with an initial guess $x_0$, and then improve the guess repeatedly ($x_0 \rightarrow x_1 \rightarrow x_2...$) until it's "close enough" to correct.  The __stopping condition__ defines when we are "close enough" in math terms.

There are Python modules that implement all of these and many more well-known algorithms for root finding.  Many are available in the [scipy.optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html#root-finding) sub-module.

One rule of good programming is: _never re-invent the wheel!_  But this is a class, so our job is to see how these algorithms work on the inside, to save you from trouble when you try to use them.

The stopping condition is usually given in terms of some __error tolerance__ $\epsilon$.  This is, ideally, the error on our numerical solution: in other words, our answer $x_i$ and the true root $x_r$ should satisfy

$$
|x_i - x_r| < \epsilon.
$$

Unfortunately, we usually can't test this directly - we don't know $x_r$!  





Instead, we can check how close $f(x_i)$ is to zero using our bound:

$$
|f(x_i) - f(x_r)| = |f(x_i)| < \epsilon.
$$

Taylor expanding about $x_r$ lets us relate this to what we want:

$$
|f(x_i)| \approx |f'(x_r)| |x_i - x_r| < \epsilon
$$

which is _almost_ the same as $|x_i - x_r| < \epsilon$; we just have an extra constant, $|f'(x_r)|$.  As long as it isn't too big, bounding $|f(x_i)|$ is a good bound on the error for estimating $x_r$.

<img src="img/root-find-sketch.png" width=500px style="float:right;margin:20px" />

Here's a more visual way to see what is going on: we want the _horizontal_ distance to the root, but the best we can usually do is the _vertical_ distance.  They are related by the slope of $f(x)$ at $x_r$.

For most practical uses, we simply __set the error tolerance to be much smaller than any other sources of error__ in our problem, so we don't care if we're off by $\epsilon$ or $2\epsilon$.  (If making $\epsilon$ really small becomes too expensive, then you need to worry about this.)

## Brute Force

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/root-brute-force.png" width=300px style="float:left;margin:20px" />

To search by brute force, we step repeatedly by $dx$ from our starting value $x_0$:

$$
x_i = x_0 + i \ dx
$$

Once we find a point which satisfies $|f(x_i)| < \epsilon$, we stop and $x_i$ is the answer.  In the plot, we go from $a$ to $b$ to $c$ to $d$, and (probably) stop at $d$.  This algorithm is fragile, because we're likely to skip past the root entirely if $\epsilon$ is too small or $dx$ is too large!

Brute force might seem crazy to you - it's very simplistic.  But it's a good introductory algorithm, and since computers are so fast, it's not ridiculous to check millions of points to find our answer!

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/bisection-sketch.png" width=500px style="float:right;" />


## Bisection method

Bisection relies on a simple math argument: if we find two points $a < b$ such that $f(a) < 0$ but $f(b) > 0$ (or vice-versa), then any smooth $f(x)$ _must cross zero_ at least once between $a$ and $b$.  We can test points in between to shrink the interval bounding where a root must exist.

In each bisection iteration we compute the midpoint $c$, and then take a new interval $[a,c]$ or $[c,b]$ - whichever one still has opposite signs of $f(x)$ on both ends.  The process repeates until the size of the interval itself is smaller than $\epsilon$.  (So this is a rare method that actually checks $|x_i - x_r|$ directly!)  

This simple algorithm converges _exponentially_ fast, since the size of the interval shrinks by a factor of 2 every time we update.

## Newton-Raphson method

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/root-newton.png" width=400px style="float:right;margin:10px" />


Also known simply as "[Newton's method](http://web.mit.edu/10.001/Web/Course_Notes/NLAE/node6.html)".  This technique uses the _derivative_ of the function as extra information: if we know both $f(x)$ and $f'(x)$, then given an initial guess $x_i$, a good guess for the nearest root is where the _tangent line_ to $f$ at $x_i$ crosses zero:

$$
x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)}
$$

This can also be proven to converge quite rapidly, but it does best in situations where we know _analytically_ what the derivative $f'(x)$ is.  (We can approximate it with samples of $f(x)$, but the convergence can be shown to be much worse.)

The square root algorithm we implemented before is actually the Newton-Raphson method!

Most root-finding algorithms converge to a __single__ root.  For functions with multiple roots, _the answer we get will tend to depend on what initial guess we give_.  The algorithm gives a __local solution__, depending on where we start.

If you need a __global solution__ (i.e. finding _all_ zeroes), you need more than a computer, since you can't test every real number!  Global information is better found with pen and paper (for example, a cubic equation has 3 or less real roots.)

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/root-finding-multi.png" width=500px style="float:left;margin:20px;" />

Choosing an initial guess is an important art for numerical solution - __always think carefully!__  Plotting your function to see roughly where any zeroes might be is often a great place to start.

The sketch shows two initial intervals $[a_1, b_1]$ and $[a_2, b_2]$ for a bisection search.  Clearly, we'll get different answers.  Moreover, there are multiple roots in $[a_1, b_1]$, so it will be hard to predict which root we find - probably not the best choice of interval!

All of my sketches so far have been well-behaved functions with roots to find.  But consider the simple $f(x) = 1/x$:

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/root-finding-pathology.png" width=450px style="float:right;"/>

If we start bisecting on either side of $x=0$, the interval will shrink until we decide $x=0$ is a root - very wrong!  (Remember, bisection requires a "smooth" function - $1/x$ definitely is not.)

Newton's method also tries to run here: since $1/x$ keeps sloping down forever, $x_i$ will run off to infinity!    This is an example of a __runaway solution__.  Runaway solutions are a good reason to have a maximum number of iterations for algorithms like this in general.

Physical quantities usually don't have divergences in them, but runaway solutions can occur even for functions that do have roots, if we choose a bad initial guess!



Root-finding algorithms need to call the function `f` that we're operating on, of course.  But we don't want to have to rewrite the root finder every single time we want to look at a new function.  

Python's flexibility to the rescue!  Since __functions are treated as data__, we can _pass a function in as an argument_ to another function:

In [ ]:
import numpy as np


def apply_and_square(f, x):
    print(f)
    return f(x) ** 2


def g(x):
    return 3 * x - 4


x = np.arange(3)
print(x)
apply_and_square(g, x)  # Pass function g as argument

Note that we are not passing the variable returned by g, but the name of the function to call later.

A root-finding function, including the ones you'll write on the tutorial, will generally take the function `f` that we're solving as an argument.

## Tutorial 15

That's it for lecture today, we'll learn this better by doing!  Connect to the server and grab assignment `tut14`.  There isn't a worked example, but we will talk through the problem setup before we start.

## Additional reading

If you're hungry for more advanced root-finding algorithms and an in-depth study of the problem in general, chapter 9 of the famous _Numerical Recipes_ has all the root-finding you can handle!  (The code examples, however, are not in Python.)